In [206]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

from skimage.filters import threshold_otsu
from skimage.morphology import closing, square

In [ ]:
video_path = "../video/rilevamento-intrusioni-video.wm"
cap = cv2.VideoCapture(video_path)

assert cap.isOpened(), "Not opened!"

fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
length = total_frame_count / fps

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"[I] Video FPS: {fps}")
print(f"[I] Video Total frame count: {total_frame_count}")
print(f"[I] Video Length: {length}")
print(f"[I] Video Frame Width: {width}")
print(f"[I] Video Frame height: {height}")

In [208]:
# Trying to understand MOG2 parameters

# # BackgroundRatio --> ok default (0.9) per ora quindi
# If a foreground pixel keeps semi-constant value for about backgroundRatio*history frames, it's considered background and added to the model 
# as a center of a new component.
# "In altre parole, TB controlla quanto velocemente il modello di sfondo si adatta ai cambiamenti nella scena."
# - Un valore più alto rende il modello di sfondo più adattabile e meno sensibile ai movimenti.
# - Un valore più basso rende il modello di sfondo più stabile e più sensibile ai movimenti, ma anche più lento ad adattarsi.

# # VarThreshold

In [209]:
# MOG2 Background Subtractor
mog2 = cv2.createBackgroundSubtractorMOG2()
mog2_test = cv2.createBackgroundSubtractorMOG2()

mog2.setHistory(fps*6)
mog2.setDetectShadows(False)
mog2.setVarThreshold(9)
mog2.setBackgroundRatio(0.6)

mog2_test.setHistory(fps)
mog2_test.setDetectShadows(False)
mog2_test.setVarThreshold(9)
mog2_test.setBackgroundRatio(0.6)

In [210]:
# """
#     Performed a sorted insertion of pv into the frame buffer in the pi row.
#     pv: pixel value
#     pi: pixel index
#     frame_buffer (global)
# """
# def sortAndInsert(pv, pi, ):
#     global frame_buffer
#     insertion_idx = np.searchsorted(frame_buffer[pi], pv)
#     if insertion_idx == len(frame_buffer[pi]):
#         insertion_idx =- 1

#     frame_buffer[pi, insertion_idx] = pv

#cv2.imshow("frame_copy", frame_contours)
# largestContour = max(contours, key = cv2.contourArea) # get largest contour
# rect = cv2.minAreaRect(largestContour)
# box = np.int64(cv2.boxPoints(rect))
# cv2.drawContours(frame_contours, [box], 0, (0,0,255), 1)
# cv2.imshow("frame_contours", frame_contours)

# cv2.imshow(f"temporalMedianBackground-frame-{frame_count}", temporalMedianBackground)
# cmd = cv2.waitKey(0)
# cv2.destroyWindow(f"temporalMedianBackground-frame-{frame_count}")
# if cmd == ord("q"):
#     break
# if cmd == ord("n"):
#     continue

# alpha = 3.0 # Contrast control
# beta = 25 # Brightness control
# # call convertScaleAbs function
# #adjusted = cv2.convertScaleAbs(mog2_fgmask, alpha=alpha, beta=beta)


# plt.hist(frame_diff.ravel(), 256, [0, 256])
# plt.show()


            
# Disegna la bounding box
#cv2.rectangle(frame_contours, (x_min, y_min), (x_max, y_max), 255, 2)

#cv2.imshow("frame_contours", frame_contours)


# analysis = cv2.connectedComponentsWithStats(tmb_fgmask, 8, cv2.CV_32S)
# (totalLabels, label_ids, values, centroid) = analysis 

# # Initialize a new image to store  
# # all the output components 
# output = np.zeros(tmb_fgmask.shape, dtype="uint8") 

# # Loop through each component 
# for i in range(1, totalLabels): 
    
#     # Area of the component 
#     area = values[i, cv2.CC_STAT_AREA]  
    
#     if (area > 140) and (area < 400): 
#         componentMask = (label_ids == i).astype("uint8") * 255
#         output = cv2.bitwise_or(output, componentMask) 

# cv2.imshow("Filtered Components", output)
# 
# 
#mog2 = cv2.createBackgroundSubtractorMOG2(history=fps*2, varThreshold=20, detectShadows=False)

# print(mog2.getHistory())
# print(mog2.getVarThreshold())
# print(mog2.getBackgroundRatio())
# print(mog2.getNMixtures())
# print(mog2.getVarThresholdGen())
# print(mog2.getVarMin())
# print(mog2.getVarMax())
# print(mog2.getComplexityReductionThreshold())
# print(mog2.getVarInit())

#mog2.setComplexityReductionThreshold(0)
#mog2.setBackgroundRatio(0.7)
#mog2.setNMixtures(1)
#mog2.setVarThresholdGen(30) 

#temporalMedianBackground_tmp = np.median(frames_tmp, axis=0).astype(dtype=np.uint8)
#cv2.imshow("temporal background - true one", temporalMedianBackground)
# make addWeighted between temporalMedianBackground and frame just where the background_mask is 1
#masked_img = np.where(background_mask[..., None] == 255, frame[..., None], 0) # dove c'è il background mask metti frame, altrimenti 0
#cv2.imshow("masked_img", masked_img)
#temporalMedianBackground = cv2.addWeighted(temporalMedianBackground, 0.7, masked_img, 0.3, 0)
#temporalMedianBackground = cv2.bitwise_and(temporalMedianBackground, temporalMedianBackground, background_mask)



In [211]:
def preprocessing(frame):
    """
        Apply all the preprocessing steps to a copy of the passed frame.

        Arguments:
            frame (MatLike): frame to process

        Returns:
            (MatLike): a processed copy of the frame
    """

    output = frame.copy()
    # Convert to grayscale
    output = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
    
    # Apply Median Blur
    output = cv2.medianBlur(output, 5)
    output = cv2.GaussianBlur(output, (7, 7), 3)

    # Apply Bilateral Filter
    output = cv2.bilateralFilter(output, 9, 75, 75)
    return output

In [212]:
def computeLearningRate(frame_count, history_length, dynamic, bias = 0):
    """
        Computes the learning rate

        Arguments:
            frame_count (int) : current frame number
            history_lenght (int): number of frames in buffer
            dynamic (boolean)
            bias (float): constant added to the computed learning rate

        Returns:
            float: the learning rate to use
        
    """
    
    assert frame_count > 0, "Frame Count must be greater than zero"
    if dynamic and frame_count < history_length:
        return (1 / frame_count) + bias
    
    return (1 / history_length) + bias

In [213]:
def postprocessing(fgmask, kernel = None):
    """
        Apply a pipeline of morphological operators to improve the segmentation

        Arguments:
            fgmask (Frame): a binary image
            kernel (Mat): kernel used by the morph operators

        Returns:
            fgmask: a processed copy of the input image
    """
    output = fgmask.copy()
    output = cv2.erode(output, None, iterations=1)
    output = cv2.dilate(output, None, iterations=3)
    output = cv2.morphologyEx(output, cv2.MORPH_CLOSE, kernel, iterations=3)
    return output

In [214]:
def backgroundSubtraction(frame, background, threshold=25, C=None):
    """
        Computes the absolute difference between the frame and the background then applies a threshold to segment the
        background and the foreground.

        Arguments:
            frame (Mat): image, current frame
            background (Mat): image, estimated reference frame
            threshold (Int)=25: if defined, is the value of the static threshold
            C (float)=None: if defined, it will use and adaptive thresholding method and it represent the value of C (Constant subtracted from the mean or weighted mean)
            
        Returns:
            fgmask: binary image with the foreground white (255).
    """
    assert C is not None or threshold is not None, "Both arguments are None!"
    diff = cv2.absdiff(frame, background)
    if C:
        block_size = 5
        fgmask = cv2.adaptiveThreshold(diff, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, C)
    else:
        _, fgmask = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    return fgmask
        

In [215]:
def getRectangularROI(fgmask):
    blur = cv2.GaussianBlur(fgmask, (5,5), 3)
    thresh = threshold_otsu(blur)
    if thresh == 0:
        return False, None
    
    bw = closing(blur >= thresh, square(7))
    #erode = cv2.erode(bw.astype(np.uint8) * 255, None, iterations=3)
    #cv2.imshow("erode", erode)
    erode = cv2.medianBlur(bw.astype(np.uint8) * 255, 3)
    dilated = cv2.dilate(erode, None, iterations=5)
    #cv2.imshow("dilate", dilated)        

    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    real_countours = []
    for contour in contours:
        # (x, y, w, h) = cv2.boundingRect(contour)
        if cv2.contourArea(contour) > 100:
            real_countours.append(contour)
        
    
    if len(real_countours) == 0:
        return False, None
    
    # Define the ROI 
    x_min = min([np.min(cnt[:, :, 0]) for cnt in real_countours])
    x_max = max([np.max(cnt[:, :, 0]) for cnt in real_countours])
    y_min = min([np.min(cnt[:, :, 1]) for cnt in real_countours])
    y_max = max([np.max(cnt[:, :, 1]) for cnt in real_countours])

    return True, (x_min, x_max, y_min, y_max)

In [ ]:
# HYPERPARAMTERS
FRAME_BUFFERED_PER_SECOND = 2           # 2 images are added to frame buffer each second
MAX_HISTORY = fps * 3                   # 3*12 frames are stored in the circual buffer
SKIP_FRAMES = fps // FRAME_BUFFERED_PER_SECOND
STATIC_THRESHOLD = 30

LEARNING_PHASE = 5 * fps              # Initialization phase for the background subtractor to estimate the reference frame

start = time.time()
frame_buffer = []
frame_count = 0
skip_count = 0
fgmask_diff = None
temporalMedianBackground = None
initialized = False
ratios = []
font = cv2.FONT_HERSHEY_SIMPLEX

kernel = np.ones((3,3),np.uint8)

while(cap.isOpened()):
    ret, frame_original = cap.read()
    if not ret or frame_original is None:
        cap.release()
        print("Released Video Resource")
        break

    frame = preprocessing(frame_original)
    frame_count += 1
    skip_count += 1

    # -------------------------------
    # |   TEMPORAL MEDIAN FILTER    |
    # -------------------------------
    if len(frame_buffer) == 0 or skip_count == SKIP_FRAMES:
        skip_count = 0
        frame_buffer.append(frame)

        if len(frame_buffer) > MAX_HISTORY:
            frame_buffer.pop(0)

        temporalMedianBackground = np.median(frame_buffer, axis=0).astype(dtype=np.uint8)
        temporalMedianBackground_copy = temporalMedianBackground.copy()
        cv2.putText(temporalMedianBackground_copy, f"FRAME: {frame_count}/{total_frame_count}", (5, 25), font, 0.5, (0, 0, 0), 1) 
        cv2.imshow("temporalMedianBackground", temporalMedianBackground_copy)

    # -------------------------------
    # |            MOG2             |
    # -------------------------------
    mog2_history = mog2.getHistory()
    learning_rate = computeLearningRate(frame_count, mog2_history, dynamic=True, bias=0.1)

    mog2_fgmask = mog2.apply(frame, learningRate=learning_rate)
    mog2_fgmask = postprocessing(mog2_fgmask)
    #cv2.imshow("Mog2 foreground mask", mog2_fgmask)
    #cv2.imshow("Eroded mog2 foreground mask", dilated)    
        
    if frame_count > LEARNING_PHASE:
        # [ GET A ROI OF THE MOG2 FOREGROUND MASK ]
        # frame_contours = frame.copy()
        found, roi = getRectangularROI(mog2_fgmask)
        # cv2.rectangle(frame_contours, (x_min, y_min), (x_max, y_max), 255, 2)
        # cv2.imshow("ROI", frame_contours)

        tmb_fgmask = backgroundSubtraction(frame, temporalMedianBackground, threshold=STATIC_THRESHOLD)
        
        # [ COMBINE FGMSKS IN ROI ]
        if found:
            combined_fgmask = mog2_fgmask.copy()
            x_min, x_max, y_min, y_max = roi
            combined_fgmask[y_min:y_max, x_min:x_max] = cv2.bitwise_or(mog2_fgmask[y_min:y_max, x_min:x_max], tmb_fgmask[y_min:y_max, x_min:x_max])    
            combined_fgmask = cv2.morphologyEx(combined_fgmask, cv2.MORPH_CLOSE, kernel, iterations=3)
            # cv2.imshow("combined_fgmask", combined_fgmask)

            # [ OVERRIDE FROZEN VALUES ]
            if fgmask_diff is not None:
                tmb_fgmask = cv2.bitwise_or(tmb_fgmask, fgmask_diff)
                #tmb_fgmask = cv2.morphologyEx(tmb_fgmask, cv2.MORPH_CLOSE, kernel, iterations=3)

                contours_fg, _ = cv2.findContours(tmb_fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                cv2.imshow("tmb_fgmask", tmb_fgmask)
                
                frame_original_contours = cv2.drawContours(frame_original, contours_fg, -1, (0,0,255), cv2.FILLED)
                cv2.imshow("frame", frame_original_contours)

                # TODO: contours and areas
                # ...


            # [ STATIC OBJECTS ]
            fgmask_diff = cv2.bitwise_and(tmb_fgmask, cv2.bitwise_not(combined_fgmask))
            #fgmask_diff = cv2.medianBlur(fgmask_diff, 3)
            #fgmask_diff = cv2.dilate(fgmask_diff, None, iterations=3)nnnnn
            # cv2.imshow("fgmask_diff", fgmask_diff)
        
        


    cmd = cv2.waitKey(0)    
    if cmd == ord("q"):
        break
    if cmd == ord("n"):
        continue
print("Mean Background ratio: ", np.mean(ratios))
# valore simile a quelo di default --> lasciamo quello di default

end = time.time()

print(f"[I] Elapsed time: {end - start} s")

cap.release()
cv2.destroyAllWindows()
